##### Author: Prashant K. Sharma

In [4]:
import pandas as pd
import numpy as np
import io, sys
import json
import ast
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import glob

In [5]:
!pwd

/home/diptesh/workspace/PR-AAAI22-SDU-ST1-AE/nbs


In [6]:
langs = ["danish", "english", "french", "persian", "spanish", "vietnamese"]

## JSON to TSVs

In [7]:
danishDFTrain = pd.read_json("../data/"+langs[0]+"/train.json")
danishDFDev = pd.read_json("../data/"+langs[0]+"/dev.json")
danishDFTest = pd.read_json("../data/"+langs[0]+"/test.json")

englishDFTrainLegal = pd.read_json("../data/"+langs[1]+"/legal/train.json")
englishDFDevLegal = pd.read_json("../data/"+langs[1]+"/legal/dev.json")
englishDFTestLegal = pd.read_json("../data/"+langs[1]+"/legal/test.json")

englishDFTrainScientific = pd.read_json("../data/"+langs[1]+"/scientific/train.json")
englishDFDevScientific = pd.read_json("../data/"+langs[1]+"/scientific/dev.json")
englishDFTestScientific = pd.read_json("../data/"+langs[1]+"/scientific/test.json")

frenchDFTrain = pd.read_json("../data/"+langs[2]+"/train.json")
frenchDFDev = pd.read_json("../data/"+langs[2]+"/dev.json")
frenchDFTest = pd.read_json("../data/"+langs[2]+"/test.json")

persianDFTrain = pd.read_json("../data/"+langs[3]+"/train.json")
persianDFDev = pd.read_json("../data/"+langs[3]+"/dev.json")
persianDFTest = pd.read_json("../data/"+langs[3]+"/test.json")

spanishDFTrain = pd.read_json("../data/"+langs[4]+"/train.json")
spanishDFDev = pd.read_json("../data/"+langs[4]+"/dev.json")
spanishDFTest = pd.read_json("../data/"+langs[4]+"/test.json")

vietnameseDFTrain = pd.read_json("../data/"+langs[5]+"/train.json")
vietnameseDFDev = pd.read_json("../data/"+langs[5]+"/dev.json")
vietnameseDFTest = pd.read_json("../data/"+langs[5]+"/test.json")

In [14]:
englishDFTestLegal

,text,ID
0,The Safe School Committee comprises of interna...,1
1,78. JS3 noted in their joint report that auth...,2
2,"In nominal pressure increments of < 1 bar, thr...",3
3,procedure (WHDC)) Proposal for the development...,4
4,10. Advocating enhanced coherence and coordin...,5
...,...,...
441,Advice and technical support to Governments in...,442
442,"Without a vision, any development effort is li...",443
443,In view of the interlinked nature of the probl...,444
444,"10, on children's rights in juvenile justice, ...",445


### Extracting text from the numerical indexes (Hadeel's Function)

In [16]:
# AUTHOR: Hadeel S

def create_acronyms(data):
    acs2 = []
    longf = []

    acros = data.acronyms.values
    txt = data.text.values
    zipped = list(zip(txt,acros))
    for z in zipped:
        acs = []
        for i in range(len(z[1])):
            acs.append(z[0][z[1][i][0]:z[1][i][1]])
        acs2.append(acs)
    data['acronyms-text'] =acs2
    
# Creating long forms from here

    longform = data["long-forms"].values
    zipped2 = list(zip(txt,longform))
    for z2 in zipped2:
        longform_temp = []
        for i in range(len(z2[1])):
            longform_temp.append(z2[0][z2[1][i][0]:z2[1][i][1]])
        longf.append(longform_temp)
    data["long-forms-text"]= longf
    
    return data
#     return data.to_csv("datatsvs/" + name +".tsv", index = False, encoding='utf8', sep="\t")

In [19]:
englishDFTrainScientific = create_acronyms(englishDFTrainScientific)
englishDFDevScientific = create_acronyms(englishDFDevScientific)
# englishDFTestScientific = create_acronyms(englishDFTestScientific)

englishDFTrainLegal = create_acronyms(englishDFTrainLegal)
englishDFDevLegal = create_acronyms(englishDFDevLegal)
# englishDFTestLegal = create_acronyms(englishDFTestLegal)

danishDFTrain = create_acronyms(danishDFTrain)
danishDFDev = create_acronyms(danishDFDev)
# danishDFTest = create_acronyms(danishDFTest)

frenchDFTrain = create_acronyms(frenchDFTrain)
frenchDFDev = create_acronyms(frenchDFDev)
# frenchDFTest = create_acronyms(frenchDFTest)

persianDFTrain = create_acronyms(persianDFTrain)
persianDFDev = create_acronyms(persianDFDev)
# persianDFTest = create_acronyms(persianDFTest)

spanishDFTrain = create_acronyms(spanishDFTrain)
spanishDFDev = create_acronyms(spanishDFDev)
# spanishDFTest = create_acronyms(spanishDFTest)

vietnameseDFTrain = create_acronyms(vietnameseDFTrain)
vietnameseDFDev = create_acronyms(vietnameseDFDev)
# vietnameseDFTest = create_acronyms(vietnameseDFTest)


### Manipulating columns for proper TSVs

In [20]:
englishDFTrainScientific = englishDFTrainScientific[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
englishDFDevScientific = englishDFDevScientific[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
englishDFTrainLegal = englishDFTrainLegal[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
englishDFDevLegal = englishDFDevLegal[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
danishDFTrain = danishDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
danishDFDev = danishDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
frenchDFTrain = frenchDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
frenchDFDev = frenchDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
persianDFTrain = persianDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
persianDFDev = persianDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
spanishDFTrain = spanishDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
spanishDFDev = spanishDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
vietnameseDFTrain = vietnameseDFTrain[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]
vietnameseDFDev = vietnameseDFDev[['ID', 'text', 'acronyms', 'long-forms', 'acronyms-text', 'long-forms-text']]

In [21]:
allLangDFTrain = englishDFTrainScientific.append(englishDFTrainLegal, ignore_index=True).append(danishDFTrain, ignore_index=True).append(frenchDFTrain, ignore_index=True).append(persianDFTrain, ignore_index=True).append(spanishDFTrain, ignore_index=True).append(vietnameseDFTrain, ignore_index=True)

In [22]:
allLangDFTrain.sample(frac=1)

,ID,text,acronyms,long-forms,acronyms-text,long-forms-text
26745,1073,"điều này , trước hết chúng ta biểu diễn logari...",[],[],[],[]
19453,1045,پايانه بايد قادر به نمونه‌برداري از ورودي‌هاي ...,"[[271, 274]]","[[226, 243]]",[SOE],[ترتيب زماني حوادث]
5053,1074,Namibia Assistance was received from the EU u...,"[[113, 118], [42, 44], [152, 155]]","[[55, 111], [127, 150]]","[TTIDP, EU, CBI]",[Transnational Trade and Investment Developmen...
11265,640,L'Institut des Nations Unies pour la formation...,"[[385, 388], [337, 343], [65, 71]]","[[350, 383], [274, 335], [2, 63]]","[OMC, UNITAR, UNITAR]","[Organisation mondiale du commerce, Institut d..."
26531,859,"HÀ NỘI , 6/2021 Đại học Bách Khoa Hà Nội",[],[],[],[]
...,...,...,...,...,...,...
26265,593,khác nhau của những bức ảnh khác nhau thì đó c...,"[[92, 94]]",[],[CL],[]
20093,349,177. Un logro importante a destacar es la sen...,"[[159, 164]]","[[124, 157]]",[ONG's],[Organizaciones no Gubernamentales]
1437,1438,TOP (PRP ? I?) ( VP (VBP ? NEED?) (,"[[17, 19], [0, 3], [5, 8]]","[[21, 24]]","[VP, TOP, PRP]",[VBP]
16414,5789,"7. En ce qui concerne les types d'activités, ...","[[166, 169]]","[[144, 164]]",[GES],[gaz à effet de serre]


### Writing TSVs

In [23]:
englishDFTestScientific

,text,ID
0,The label restricted means the model is restri...,1
1,We have followed their methodology as best as ...,2
2,adaptation scenario. Duan et al (2009) propose...,3
3,Actes de la 13e Confe?rence sur le Traitement...,4
4,"TGTM P=p,pk ,b TGTM PR=pr , pkr , b r TG...",5
...,...,...
493,measured on separate grammatical and ungrammat...,494
494,Schu?tze reduces the dimensionality of this fe...,495
495,"is a.t least cubic in t, ime, this fl)llows tr...",496
496,"present specialized knowledge, since both the ...",497


In [24]:
englishDFTrainScientific.to_csv("../data_tsvs/englishDFTrainScientific.tsv", sep='\t', index=False)
englishDFDevScientific.to_csv("../data_tsvs/englishDFDevScientific.tsv", sep='\t', index=False)
englishDFTestScientific.to_csv("../data_tsvs/englishDFTestScientific.tsv", sep='\t', index=False)

englishDFTrainLegal.to_csv("../data_tsvs/englishDFTrainLegal.tsv", sep='\t', index=False)
englishDFDevLegal.to_csv("../data_tsvs/englishDFDevLegal.tsv", sep='\t', index=False)
englishDFTestLegal.to_csv("../data_tsvs/englishDFTestLegal.tsv", sep='\t', index=False)

danishDFTrain.to_csv("../data_tsvs/danishDFTrain.tsv", sep='\t', index=False)
danishDFDev.to_csv("../data_tsvs/danishDFDev.tsv", sep='\t', index=False)
danishDFTest.to_csv("../data_tsvs/danishDFTest.tsv", sep='\t', index=False)

frenchDFTrain.to_csv("../data_tsvs/frenchDFTrain.tsv", sep='\t', index=False)
frenchDFDev.to_csv("../data_tsvs/frenchDFDev.tsv", sep='\t', index=False)
frenchDFTest.to_csv("../data_tsvs/frenchDFTest.tsv", sep='\t', index=False)

persianDFTrain.to_csv("../data_tsvs/persianDFTrain.tsv", sep='\t', index=False)
persianDFDev.to_csv("../data_tsvs/persianDFDev.tsv", sep='\t', index=False)
persianDFTest.to_csv("../data_tsvs/persianDFTest.tsv", sep='\t', index=False)

spanishDFTrain.to_csv("../data_tsvs/spanishDFTrain.tsv", sep='\t', index=False)
spanishDFDev.to_csv("../data_tsvs/spanishDFDev.tsv", sep='\t', index=False)
spanishDFTest.to_csv("../data_tsvs/spanishDFTest.tsv", sep='\t', index=False)

vietnameseDFTrain.to_csv("../data_tsvs/vietnameseDFTrain.tsv", sep='\t', index=False)
vietnameseDFDev.to_csv("../data_tsvs/vietnameseDFDev.tsv", sep='\t', index=False)
vietnameseDFTest.to_csv("../data_tsvs/vietnameseDFTest.tsv", sep='\t', index=False)

In [25]:
allLangDFTrain.to_csv("../data_tsvs/allLangDFTrain.tsv", sep='\t', index=False)

# TSVs to CoNLL format 

In [26]:
def BIO(txt, an_ids, lf_ids):
    """
    input: text
    ouput: tags , bio_tags
        
        tags :    list (AN, FL, OT)
        bio_tags: list (O, B-AN, I-AN, B-LF, I-LF)
    """
    
    txt_l = word_tokenize(txt)
    
    tags = ["OT"]*len(txt_l)
    bio_tags = ["B-O"]*len(txt_l)
    
    #txt[an_ids[0][0]:an_ids[0][1]]
    
    ANs = []
    LFs = []
    
    for an_id in an_ids:
        ANs.append(txt[an_id[0]:an_id[1]])
        
    for lf_id in lf_ids:
        LFs.append(txt[lf_id[0]:lf_id[1]])
    
    AN_start = [False]*len(ANs)
    LF_start = [False]*len(LFs)
    
    for tok_id in range(0,len(txt_l)):
        tok = txt_l[tok_id]
        
        for AN_idx in range(0,len(ANs)):
           
            AN = ANs[AN_idx]
            
            if(tok in AN):
                tags[tok_id] = "AN"
                
                if(AN_start[AN_idx] == False):
                    AN_start[AN_idx] = True
                    bio_tags[tok_id] = "B-AN"
                else:
                    bio_tags[tok_id] = "I-AN"
        
        for LF_idx in range(0,len(LFs)):
            
            LF = LFs[LF_idx]
            
            if(tok in LF):
                tags[tok_id] = "LF"
                
                if(LF_start[LF_idx] == False):
                    LF_start[LF_idx] = True
                    bio_tags[tok_id] = "B-LF"
                else:
                    bio_tags[tok_id] = "I-LF"
                    
    return txt_l, tags, bio_tags

In [27]:
def extract(df, idx):
    df_i = df.iloc[idx]
    
    txt = str(df_i["text"]).strip()
    acrnm_ids = ast.literal_eval(df_i["acronyms"])
    longform_ids = ast.literal_eval(df_i["long-forms"])
    
    return txt, acrnm_ids, longform_ids

def prepare(df, cnt, fname="output.txt"):
    
    file = open(fname, "w")
    file.write("-DOCSTART- -X- O\n\n")
    for idx in tqdm(range(cnt)):
        txt, an_ids, lf_ids = extract(df, idx)
        
        txt_l, tags, bio_tags = BIO(txt, an_ids, lf_ids)
        
        for txt_id in range(0,len(txt_l)):
            string = txt_l[txt_id]+ " " + tags[txt_id] + " " + bio_tags[txt_id]
            
            file.write(string + " \n")
        file.write("\n")
    file.close()
    
def convert_to_BIO(fileTrain, fileDev, lang, domain):
    df_train = pd.read_csv(fileTrain, sep="\t")
    df_dev = pd.read_csv(fileDev, sep="\t")
    df_combine = df_train.append(df_dev, ignore_index=True)
    
    cnt_train = df_train.shape[0]
    cnt_dev = df_dev.shape[0]
    
    train_cnt = cnt_train - 1000 ## Assuming 300 instances are enough to validate
    valid_cnt = 1000
    test_cnt = cnt_dev
    
    df_train_ = df_combine.iloc[:train_cnt]
    df_valid_ = df_combine.iloc[train_cnt: train_cnt + valid_cnt].reset_index()
    df_test_ = df_combine.iloc[train_cnt + valid_cnt: train_cnt + valid_cnt + test_cnt].reset_index()
    
    # Prepare training file
    if ( domain == "" ):
        prepare(df_train_, train_cnt, "../processed/" + lang + "/train.txt") # Prepare train file
        prepare(df_valid_, valid_cnt, "../processed/" + lang + "/valid.txt") # Prepare validation file
        prepare(df_test_, test_cnt, "../processed/" + lang + "/test.txt") # Prepare test file
    else:
        prepare(df_train_, train_cnt, "../processed/" + lang + "/" + domain + "/train.txt") # Prepare train file
        prepare(df_valid_, valid_cnt, "../processed/" + lang + "/" + domain +"/valid.txt") # Prepare validation file
        prepare(df_test_, test_cnt, "../processed/" + lang+  "/" + domain + "/test.txt") # Prepare test file

In [60]:
PLOSDFTrain = pd.read_csv("../data_tsvs/PLOS-train-data.tsv", sep="\t", header=None)
PLOSDFTrain = PLOSDFTrain.drop(columns=[6])
# allLangDFTrain.append(englishDFTrainLegal, ignore_index=True)

In [61]:
PLOSDFTrain.columns = allLangDFTrain.columns
allLangDFTrain = allLangDFTrain.append(PLOSDFTrain, ignore_index=True)
allLangDFTrain = allLangDFTrain.sample(frac=1)
allLangDFTrain.to_csv("../data_tsvs/allLangPLOSDFTrain_1000.tsv", sep="\t", index=False)

In [62]:
convert_to_BIO("../data_tsvs/englishDFTrainLegal.tsv", "../data_tsvs/englishDFDevLegal.tsv", "eng", "legal")
convert_to_BIO("../data_tsvs/englishDFTrainScientific.tsv", "../data_tsvs/englishDFDevScientific.tsv", "eng", "scientific")
convert_to_BIO("../data_tsvs/danishDFTrain.tsv", "../data_tsvs/danishDFDev.tsv", "dan", "")
convert_to_BIO("../data_tsvs/frenchDFTrain.tsv", "../data_tsvs/frenchDFDev.tsv", "fre", "")
convert_to_BIO("../data_tsvs/persianDFTrain.tsv", "../data_tsvs/persianDFDev.tsv", "per", "")
convert_to_BIO("../data_tsvs/spanishDFTrain.tsv", "../data_tsvs/spanishDFDev.tsv", "esp", "")
convert_to_BIO("../data_tsvs/vietnameseDFTrain.tsv", "../data_tsvs/vietnameseDFDev.tsv", "vie", "")        

100%|██████████| 159/159 [00:00<00:00, 2547.03it/s]


In [63]:
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/englishDFDevLegal.tsv", "engAll", "legal")
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/englishDFDevScientific.tsv", "engAll", "scientific")
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/danishDFDev.tsv", "danAll", "")
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/frenchDFDev.tsv", "freAll", "")
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/persianDFDev.tsv", "perAll", "")
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/spanishDFDev.tsv", "espAll", "")
# convert_to_BIO("../data_tsvs/allLangDFTrain.tsv", "../data_tsvs/vietnameseDFDev.tsv", "vieAll", "") 

In [65]:
convert_to_BIO("../data_tsvs/allLangPLOSDFTrain_1000.tsv", "../data_tsvs/englishDFDevScientific.tsv", "engAllPLOS_1000", "scientific")

100%|██████████| 497/497 [00:00<00:00, 2694.86it/s]
